### Importaciones

In [ ]:
import os
from dotenv import load_dotenv
from urllib.parse import quote
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext

In [2]:
load_dotenv(override=True)

True

### Variables de entorno

In [3]:
sharepoint_user = os.getenv('SHAREPOINT_USER')
sharepoint_password = os.getenv('SHAREPOINT_PASSWORD')
sharepoint_site_url = os.getenv('SHAREPOINT_SITE_URL')
sharepoint_dossier_url = os.getenv('SHAREPOINT_DOSSIER_URL')

### Clientes

In [4]:
ctx = ClientContext(sharepoint_site_url).with_credentials(
    UserCredential(sharepoint_user, sharepoint_password)
)

### Funciones

In [5]:
# --- util: obtener carpeta (manejo de espacios) ---
def get_folder(ctx, server_relative_url: str):
    try:
        folder = ctx.web.get_folder_by_server_relative_url(server_relative_url)
        ctx.load(folder)
        ctx.execute_query()
        return folder
    except Exception:
        encoded = quote(server_relative_url)
        folder = ctx.web.get_folder_by_server_relative_url(encoded)
        ctx.load(folder)
        ctx.execute_query()
        return folder

# --- recorrido recursivo que devuelve SOLO archivos (rutas relativas) ---
dossier_base = sharepoint_dossier_url.rstrip("/")
files_rel = []

def walk_files(ctx, folder):
    folder.expand(["Folders", "Files"]).get().execute_query()

    # agregar archivos de esta carpeta
    for f in folder.files:
        srv = f.properties.get("ServerRelativeUrl", "")
        rel = srv[len(dossier_base):].lstrip("/")  # quitar prefijo del dossier y la primera barra
        files_rel.append(rel)

    # recorrer subcarpetas
    for sub in folder.folders:
        walk_files(ctx, sub)

root = get_folder(ctx, sharepoint_dossier_url)
walk_files(ctx, root)

# ordenar para salida estable
files_rel.sort(key=lambda p: p.lower())

# --- escribir markdown plano ---
os.makedirs("output", exist_ok=True)
out_path = os.path.join("output", "dossier_estructura.md")

with open(out_path, "w", encoding="utf-8") as f:
    f.write("Estructura del dossier\n")
    f.write(f"Raíz: {dossier_base}\n")
    for p in files_rel:
        f.write(f"- {p}\n")

print(f"Generado: {out_path} ({len(files_rel)} archivos)")

Generado: output\dossier_estructura.md (158 archivos)
